In [14]:
# Data import
import pyomo.environ as pe
import math

# Data
factories = ["IJmuiden", "Segal", "South Wales"]
rebars = {"A": {"length": 2.4}, "B": {"length": 3.6}, "C": {"length": 4.2}}
longbars = {1: {"length": 9}, 2: {"length": 12}}
diameter = 0.057  # in meters
density = 7.85  # tons/m^3
production_capacity = {"IJmuiden": 12, "Segal": 10, "South Wales": 28}
customers = ["Bochum", "Boenen", "Dortmund", "Gelsenkirchen", "Hagen", "Iserlohn", "Neuss", "Schwerte"]
periods = [1, 2, 3, 4]

demand = {
    "A": {
        "Bochum": [2, 6, 5, 3],
        "Boenen": [4, 8, 5, 10],
        "Dortmund": [2, 7, 6, 5],
        "Gelsenkirchen": [5, 5, 5, 5],
        "Hagen": [19, 23, 25, 16],
        "Iserlohn": [13, 19, 17, 14],
        "Neuss": [20, 16, 14, 26],
        "Schwerte": [4, 5, 3, 4]
    },
    "B": {
        "Bochum": [4, 5, 7, 8],
        "Boenen": [5, 8, 12, 13],
        "Dortmund": [4, 5, 8, 10],
        "Gelsenkirchen": [9, 10, 6, 6],
        "Hagen": [15, 33, 31, 33],
        "Iserlohn": [22, 26, 20, 27],
        "Neuss": [12, 23, 30, 30],
        "Schwerte": [2, 8, 2, 6]
    },
    "C": {
        "Bochum": [6, 7, 7, 7],
        "Boenen": [6, 10, 15, 12],
        "Dortmund": [7, 6, 4, 12],
        "Gelsenkirchen": [10, 9, 9, 10],
        "Hagen": [12, 35, 33, 38],
        "Iserlohn": [14, 25, 23, 24],
        "Neuss": [22, 32, 31, 31],
        "Schwerte": [5, 6, 7, 2]
    }
}

fixed_cost = {"IJmuiden": 130, "Segal": 150, "South Wales": 100}
v = 0.5  # variable cost €/km/tonne
distance = {
    "Bochum": {"IJmuiden": 250, "Segal": 203, "South Wales": 866},
    "Boenen": {"IJmuiden": 282, "Segal": 242, "South Wales": 914},
    "Dortmund": {"IJmuiden": 266, "Segal": 222, "South Wales": 885},
    "Gelsenkirchen": {"IJmuiden": 234, "Segal": 198, "South Wales": 859},
    "Hagen": {"IJmuiden": 289, "Segal": 206, "South Wales": 903},
    "Iserlohn": {"IJmuiden": 299, "Segal": 226, "South Wales": 913},
    "Neuss": {"IJmuiden": 259, "Segal": 140, "South Wales": 843},
    "Schwerte": {"IJmuiden": 279, "Segal": 216, "South Wales": 901}
}

# Precompute parameters
w_r = {r: rebars[r]["length"] for r in rebars}
w_l = {l: longbars[l]["length"] for l in longbars}
area = math.pi * (diameter / 2) ** 2


# possible number of long bars per long bar type per factory 
N = {
    1: {"IJmuiden": 67, "Segal": 56, "South Wales": 156},
    2: {"IJmuiden": 50, "Segal": 42, "South Wales": 117}
}

possible_longbars = {} 

for l in longbars:
    possible_longbars[l] = {} 
    for f in factories:
        possible_longbars[l][f] = list(range(1, N.get(l, {}).get(f, 0) + 1))

# Defining big M
# List to store total demand per customer
total_demand_per_customer = []

# Loop over each customer
for a in customers:
    total_demand = sum(demand[r][a][t-1] for r in rebars for t in periods)  # Sum over all rebars and periods
    total_demand_per_customer.append(total_demand)

# Get the maximum demand across all customers
M = max(total_demand_per_customer)

inventory_capacity = {
    "Bochum": 10,
    "Boenen": 7,
    "Dortmund": 12,
    "Gelsenkirchen": 10,
    "Hagen": 12,
    "Iserlohn": 9,
    "Neuss": 8,
    "Schwerte": 5
}

  

In [15]:
#New data for 3: 
distanceq3 = {
    "IJmuiden": {"IJmuiden": 0, "Segal": 284, "South Wales": 826, "Bochum": 250, "Boenen": 282, "Dortmund": 266, "Gelsenkirchen": 234, "Hagen": 289, "Iserlohn": 299, "Neuss": 259, "Schwerte": 279},
    "Segal": {"IJmuiden": 284, "Segal": 0, "South Wales": 750, "Bochum": 203, "Boenen": 242, "Dortmund": 222, "Gelsenkirchen": 198, "Hagen": 206, "Iserlohn": 226, "Neuss": 140, "Schwerte": 216},
    "South Wales": {"IJmuiden": 826, "Segal": 750, "South Wales": 0, "Bochum": 866, "Boenen": 914, "Dortmund": 885, "Gelsenkirchen": 859, "Hagen": 903, "Iserlohn": 913, "Neuss": 843, "Schwerte": 901},
    "Bochum": {"IJmuiden": 250, "Segal": 203, "South Wales": 866, "Bochum": 0, "Boenen": 55, "Dortmund": 21, "Gelsenkirchen": 19, "Hagen": 41, "Iserlohn": 51, "Neuss": 56, "Schwerte": 39},
    "Boenen": {"IJmuiden": 282, "Segal": 242, "South Wales": 914, "Bochum": 55, "Boenen": 0, "Dortmund": 34, "Gelsenkirchen": 56, "Hagen": 44, "Iserlohn": 54, "Neuss": 102, "Schwerte": 32},
    "Dortmund": {"IJmuiden": 266, "Segal": 222, "South Wales": 885, "Bochum": 21, "Boenen": 34, "Dortmund": 0, "Gelsenkirchen": 34, "Hagen": 21, "Iserlohn": 36, "Neuss": 75, "Schwerte": 15},
    "Gelsenkirchen": {"IJmuiden": 234, "Segal": 198, "South Wales": 859, "Bochum": 19, "Boenen": 56, "Dortmund": 34, "Gelsenkirchen": 0, "Hagen": 56, "Iserlohn": 66, "Neuss": 62, "Schwerte": 54},
    "Hagen": {"IJmuiden": 289, "Segal": 206, "South Wales": 903, "Bochum": 41, "Boenen": 44, "Dortmund": 21, "Gelsenkirchen": 56, "Hagen": 0, "Iserlohn": 20, "Neuss": 66, "Schwerte": 14},
    "Iserlohn": {"IJmuiden": 299, "Segal": 226, "South Wales": 913, "Bochum": 51, "Boenen": 54, "Dortmund": 36, "Gelsenkirchen": 66, "Hagen": 20, "Iserlohn": 0, "Neuss": 85, "Schwerte": 15},
    "Neuss": {"IJmuiden": 259, "Segal": 140, "South Wales": 843, "Bochum": 56, "Boenen": 102, "Dortmund": 75, "Gelsenkirchen": 62, "Hagen": 66, "Iserlohn": 85, "Neuss": 0, "Schwerte": 75},
    "Schwerte": {"IJmuiden": 279, "Segal": 216, "South Wales": 901, "Bochum": 39, "Boenen": 32, "Dortmund": 15, "Gelsenkirchen": 54, "Hagen": 14, "Iserlohn": 15, "Neuss": 75, "Schwerte": 0}
}

#Vehicle capacity
Q = 25
# Cost per km
C = 3

model3 = pe.ConcreteModel()

# Initialize sets for model 2
model3.F = pe.Set(initialize=factories)
model3.R = pe.Set(initialize=rebars.keys())
model3.L = pe.Set(initialize=longbars.keys())
model3.A = pe.Set(initialize=customers)
model3.T = pe.Set(initialize=periods)
model3.F_A = model3.F | model3.A
model3.A_f = pe.Set(model3.F, initialize=lambda model3, f: model3.A | {f})
#model3.C = pe.Set(model3.A, model3.T, within=model3.A, initialize=lambda model, a, t: [])


valid_x_indices = []
for l in possible_longbars:
    for f in possible_longbars[l]:
        for n in possible_longbars[l][f]:  # Only valid values of n
            valid_x_indices.append((l, n, f))

# Create an indexed set for (L, F, N)
model3.L_N_F = pe.Set(dimen=3, initialize=valid_x_indices)

# Define variables
model3.x = pe.Var(model3.R, model3.L_N_F,  model3.T, domain=pe.NonNegativeIntegers)
model3.y = pe.Var(model3.L_N_F, model3.T, domain=pe.Binary)
#model3.z = pe.Var(model3.F, model3.A, model3.T, domain=pe.Binary, initialize = 0)
model3.i = pe.Var(model3.A, model3.R, model3.T, domain=pe.NonNegativeIntegers)
model3.q = pe.Var(model3.R, model3.F, model3.A, model3.T, domain=pe.NonNegativeIntegers, initialize = 0)

# Define the valid indices for model3.v dynamically
#valid_v_indices = [(f, A_f, A_f, t) for f in model3.F for t in model3.T for A_f in model3.A_f[f]]
valid_v_indices = [
    (f, A_f_source, A_f_dest, t)
    for f in model3.F
    for t in model3.T
    for A_f_source in model3.A_f[f]
    for A_f_dest in model3.A_f[f]
]

# Add new variables to model3
model3.v = pe.Var(valid_v_indices, domain=pe.Binary, initialize = 0)
model3.u = pe.Var(model3.A, model3.F, model3.T, domain=pe.NonNegativeIntegers, initialize = 0)

# Add notation for t_fat and z_fat for simplicity
# Define t_fat as the tonnes shipped from factory f to customer a in period t
model3.t = pe.Expression(model3.F, model3.A, model3.T, 
                                  rule=lambda model3, f, a, t: 
                                  sum(model3.q[(r, f, a, t)] * w_r[r] * density * area for r in model3.R))
      
model3.z = pe.Expression(model3.F, model3.A, model3.T, 
                                  rule=lambda model3, f, a, t: 
                                  sum(model3.v[(f, i, a, t)] for i in model3.A_f[f]))
  

# Constraints 
# 1. Cutting constraints - same as Q1
model3.cutting_cnstr = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        for l in model3.L:
            for n in possible_longbars[l][f]:
                expression = sum(model3.x[r, l, n, f, t] * w_r[r] for r in model3.R) <= model3.y[l, n, f, t] * w_l[l]
                model3.cutting_cnstr.add(expression)

# 2. Production capacity - same as Q1
model3.capacity_cnstr = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        expression = sum(model3.y[l, n, f, t] * w_l[l] for n in possible_longbars[l][f] for l in model3.L) * density * area <= production_capacity[f]
        model3.capacity_cnstr.add(expression)

# Factory output - same as Q2
model3.factory_output = pe.ConstraintList()
for f in model3.F:
    for r in model3.R:
        for t in model3.T:
            expression = sum(model3.q[(r, f, a, t)] for a in model3.A) == sum(model3.x[(r, l, n, f, t)] for n in possible_longbars[l][f] for l in model3.L)
            model3.factory_output.add(expression)

# Link z to q - same as Q2
model3.enforce_factory_usage = pe.ConstraintList()
for f in model3.F:
    for a in model3.A:
        for t in model3.T:
            expression = sum(model3.q[(r, f, a, t)] for r in model3.R) <= M * model3.z[f, a, t]
            model3.enforce_factory_usage.add(expression)
            
#  production triggers customer assignment constraint (link y to z) - same as Q1
model3.prod_triggers_customer_assignment = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        # Sum over all r, l, n as specified
        expression = sum(model3.y[ l, n, f, t]
                         for l in model3.L 
                         for n in possible_longbars[l][f]) <= M * sum(model3.z[f, a, t] for a in model3.A)
        
        # Add the constraint to the model
        model3.prod_triggers_customer_assignment.add(expression)


# Demand - same as Q2
model3.demand = pe.ConstraintList()
for a in model3.A:
    for r in model3.R:
        for t in model3.T:
            shipped = sum(model3.q[(r, f, a, t)] for f in model3.F) 
            if t == 1:
                expression = shipped - model3.i[(a,r,t)] == demand[r][a][t-1] 
                model3.demand.add(expression)
            else:
                expression = shipped - model3.i[(a,r,t)] + model3.i[(a,r,t-1)] == demand[r][a][t-1] 
                model3.demand.add(expression)

# Single sourcing - factory assignment - same as Q2
model3.single_sourcing_cnstr = pe.ConstraintList()
for a in model3.A:
    for t in model3.T:
        expression = sum(model3.z[(f,a,t)] for f in model3.F) <= 1
        model3.single_sourcing_cnstr.add(expression)

# Inventory - same as Q2
model3.inventory = pe.ConstraintList()
for a in model3.A:
    for t in model3.T:
        expression = sum(model3.i[(a,r,t)]* w_r[r] for r in model3.R) * density * area <= inventory_capacity[a]
        model3.inventory.add(expression)

# New constraints
                              
# Flow conservation
model3.flow_conservation = pe.ConstraintList()
for f in model3.F:
    for j in model3.A_f[f]:
        for t in model3.T:
            expression = sum(model3.v[(f,i,j,t)] for i in model3.A_f[f]) == sum(model3.v[(f,j,h,t)] for h in model3.A_f[f])
            model3.flow_conservation.add(expression)

# Subtour Elimination Constraint
model3.subtour_elimination = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        for i in model3.A: 
            for j in model3.A:
                if i != j:
                    expression = model3.u[i,f,t] + model3.t[f,j,t] <= model3.u[j,f,t] + Q * (1 - model3.v[(f,i, j, t)])
                    model3.subtour_elimination.add(expression)

# Limiting t
model3.upper_bound_t = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        for i in model3.A:
            expression = model3.t[f,i,t] <= model3.u[i,f,t]
            model3.upper_bound_t.add(expression)

# don't go from i to i
model3.prevent_loops = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        for i in model3.A_f[f]:
            expression = model3.v[(f,i, i, t)] == 0
            model3.prevent_loops.add(expression)


# Limiting u
model3.upper_bound_u = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        for i in model3.A:
            expression = model3.u[i,f,t] <= Q
            model3.upper_bound_u.add(expression)

# Vehicle capacity
model3.vehicle_capacity = pe.ConstraintList()
for f in model3.F:
    for t in model3.T:
        expression = sum(model3.t[f,a,t] for a in model3.A) <= Q
        model3.vehicle_capacity.add(expression)


# Objective function
objExpr = sum(model3.z[f, a, t] * C * (sum(model3.v[(f,i, a, t)] * distanceq3.get(i, {}).get(a, 0) for i in model3.A_f[f]) + model3.v[(f,a, f, t)] * distanceq3.get(a, {}).get(f, 0) )for f in model3.F for a in model3.A for t in model3.T)
model3.obj = pe.Objective(expr=objExpr, sense=pe.minimize)

# Solve the model
solver = pe.SolverFactory('gurobi')
result3 = solver.solve(model3, tee=True, options={'TimeLimit':3600})

print(f"\nSolver Status: {result3.solver.status}")
print(f"Termination Condition: {result3.solver.termination_condition}")
print(f"Objective value (Total Cost): {pe.value(model3.obj):.2f} €")

#debugging
for f in model3.F:
    for a in model3.A:
        for t in model3.T:
            for r in model3.R:
                if pe.value(model3.q[r, f, a, t]) > 0:
                    print(f"Rebar {r} is shipped from {f} to {a} in period {t} with quantity {pe.value(model3.q[r, f, a, t])}")

for f in model3.F:
    for a in model3.A:
        for t in model3.T:
            if pe.value(model3.z[f, a, t]) > 0:
                print(f"Factory {f} is assigned to customer {a} in period {t}")


Set parameter Username
Set parameter LicenseID to value 2618603
Academic license - for non-commercial use only - expires 2026-02-05
Read LP format model from file /var/folders/dh/_xb_27wd2qg4j5p7s0bnfz1w0000gn/T/tmp3_nlyl09.pyomo.lp
Reading time = 0.19 seconds
x1: 3264 rows, 9260 columns, 26420 nonzeros
Set parameter TimeLimit to value 3600
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (mac64[x86] - Darwin 23.6.0 23H417)

CPU model: Intel(R) Core(TM) i5-8210Y CPU @ 1.60GHz
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads

Non-default parameters:
TimeLimit  3600

Optimize a model with 3264 rows, 9260 columns and 26420 nonzeros
Model fingerprint: 0xf2b05209
Model has 5088 quadratic objective terms
Variable types: 0 continuous, 9260 integer (2924 binary)
Coefficient statistics:
  Matrix range     [5e-02, 3e+02]
  Objective range  [0e+00, 0e+00]
  QObjective range [8e+01, 6e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+01]
Presolve remov

In [18]:
# Print all values of v (binary decision variable)
for f in model3.F:
    for i in model3.A_f[f]:
        for j in model3.A_f[f]:
            for t in model3.T:
                v_value = pe.value(model3.v[(f, i, j, t)])
                if v_value > 0:
                    print(f"v[{f}, {i}, {j}, {t}] = {v_value}")


v[IJmuiden, Hagen, IJmuiden, 1] = 1.0
v[IJmuiden, IJmuiden, Hagen, 1] = 1.0
v[Segal, Neuss, Segal, 1] = 1.0
v[Segal, Segal, Neuss, 1] = 1.0
v[South Wales, Bochum, Dortmund, 1] = 1.0
v[South Wales, Bochum, South Wales, 2] = 1.0
v[South Wales, Boenen, Gelsenkirchen, 1] = 1.0
v[South Wales, Dortmund, Bochum, 2] = 1.0
v[South Wales, Dortmund, Iserlohn, 1] = 1.0
v[South Wales, Gelsenkirchen, South Wales, 1] = 1.0
v[South Wales, Hagen, South Wales, 3] = 1.0
v[South Wales, Iserlohn, Schwerte, 1] = 1.0
v[South Wales, Iserlohn, Schwerte, 2] = 1.0
v[South Wales, Neuss, Hagen, 3] = 1.0
v[South Wales, Schwerte, Boenen, 1] = 1.0
v[South Wales, Schwerte, Dortmund, 2] = 1.0
v[South Wales, South Wales, Bochum, 1] = 1.0
v[South Wales, South Wales, Iserlohn, 2] = 1.0
v[South Wales, South Wales, Neuss, 3] = 1.0


In [16]:
# After solving the model

# Print the routes for each factory and the customers visited in each period

for t in model3.T:
    for f in model3.F:
        print(f"\nFactory {f} - Period {t}:")
        visited_customers = []
        
        # Identify the customers visited by this factory in period t
        for i in model3.A_f[f]:
            if pe.value(model3.v[(f, f, i, t)]) > 0.5:
                next_customer = i
                visited_customers.append(i)
                while next_customer != f:
                    for j in model3.A_f[f]:
                        if i != j and pe.value(model3.v[(f, next_customer, j, t)]) > 0.5:  # if v[(f, i, j, t)] is 1
                            next_customer = j
                            visited_customers.append(next_customer)

        
        # Now print the route (start -> customer visits -> end)
        if visited_customers:
            route_str = f"Start -> "
            route_str += " -> ".join([f"{i}" for i in visited_customers])
            route_str += " -> End"
            print(route_str)
        else:
            print("No route assigned (no customers visited).")



Factory IJmuiden - Period 1:
Start -> Hagen -> IJmuiden -> End

Factory Segal - Period 1:
Start -> Neuss -> Segal -> End

Factory South Wales - Period 1:
Start -> Bochum -> Dortmund -> Iserlohn -> Schwerte -> Boenen -> Gelsenkirchen -> South Wales -> End

Factory IJmuiden - Period 2:
No route assigned (no customers visited).

Factory Segal - Period 2:
No route assigned (no customers visited).

Factory South Wales - Period 2:
Start -> Iserlohn -> Schwerte -> Dortmund -> Bochum -> South Wales -> End

Factory IJmuiden - Period 3:
No route assigned (no customers visited).

Factory Segal - Period 3:
No route assigned (no customers visited).

Factory South Wales - Period 3:
Start -> Neuss -> Hagen -> South Wales -> End

Factory IJmuiden - Period 4:
No route assigned (no customers visited).

Factory Segal - Period 4:
No route assigned (no customers visited).

Factory South Wales - Period 4:
No route assigned (no customers visited).


In [19]:
# Analysis of production and demand

total_demand_per_period = {1: 0, 2: 0, 3: 0, 4: 0}
for r in rebars:
    for customer in demand[r]:
        for t in range(4):  # For each period
            # Demand in tonnes for each rebar type, customer, and period
            demand_in_tonnes = demand[r][customer][t] * w_r[r] * density * area
            total_demand_per_period[t + 1] += demand_in_tonnes

# Initialize a dictionary to store production data
factory_production = {f: {t: 0 for t in range(1, 5)} for f in ["IJmuiden", "Segal", "South Wales"]}
total_production_per_period = {t: 0 for t in range(1, 5)}  # Total production for each period

# Extract production values from model3
for f in factory_production:
    for t in factory_production[f]:
        total_tonnes = sum(
            pe.value(model3.y[l, n, f, t]) * w_l[l] * density * area
            for l in model3.L for n in possible_longbars[l][f]
            if pe.value(model3.y[l, n, f, t]) > 0.5
        )
        factory_production[f][t] = round(total_tonnes, 2)  # Round to 2 decimals
        total_production_per_period[t] += total_tonnes  # Update total production per period

# Print table
print("Factory Production (Tonnes per Period)")
print("-" * 80)
print(f"{'Factory':<15} {'P1':<10} {'P2':<10} {'P3':<10} {'P4':<10} ")
print("-" * 80)

# Print the factory production values
for f, periods in factory_production.items():
    row = f"{f:<15}"
    for t in range(1, 5):
        row += f"{periods[t]:<10}"
    print(row)

# Print total production across all factories
row = f"{'Total Production':<15}"
for t in range(1, 5):
    row += f"{total_production_per_period[t]:<10}"
row += f"{round(sum(total_production_per_period.values()), 2):<15.2f}"  # Rounded total production
print(row)

# Print total demand per period
row = f"{'Total Demand':<15}"
for t in total_demand_per_period:
    row += f"{round(total_demand_per_period[t], 2):<10}"
print(row)

print()

# Cost per period

total_cost_per_period = {t: 0 for t in model3.T}  # Dictionary to store total cost for each period

# Loop through each period, customer, and factory to calculate the total cost per period
for t in model3.T:
    period_cost = 0  # Initialize the cost for the current period
    for f in model3.F:
        for a in model3.A:
            # Calculate the cost of transportation and fixed costs for each factory, customer, and period
            
            period_cost += pe.value(model3.z[f, a, t]) * (
            C * (sum(pe.value(model3.v[(f,i, a, t)]) * distanceq3.get(i, {}).get(a, 0) for i in model3.A_f[f]) + pe.value(model3.v[(f,a, f, t)]) * distanceq3.get(a, {}).get(f, 0) )
            )
    # Store the total cost for this period
    total_cost_per_period[t] = period_cost

# Print the total cost per period
for t in total_cost_per_period:
    print(f"Total cost for Period {t}: €{total_cost_per_period[t]:.2f}")


Factory Production (Tonnes per Period)
--------------------------------------------------------------------------------
Factory         P1         P2         P3         P4         
--------------------------------------------------------------------------------
IJmuiden       11.96     0         0         0         
Segal          9.86      0         0         0         
South Wales    26.5      25.24     25.0      0         
Total Production48.3155083066444625.2394446377993424.9990689745822050         98.55          
Total Demand   15.48     23.73     23.06     25.02     

Total cost for Period 1: €8229.00
Total cost for Period 2: €5490.00
Total cost for Period 3: €5436.00
Total cost for Period 4: €0.00


In [15]:
f = "South Wales"
t = 3
total_shipped = 0
for a in model3.A:
    total_shipped += pe.value(model3.t[f,a,t])
print(total_shipped)

22.60733112557171


In [4]:
print(f"Objective Function Expression: {model3.obj.expr}")


Objective Function Expression: (v[IJmuiden,Bochum,Bochum,1] + v[IJmuiden,Boenen,Bochum,1] + v[IJmuiden,Dortmund,Bochum,1] + v[IJmuiden,Gelsenkirchen,Bochum,1] + v[IJmuiden,Hagen,Bochum,1] + v[IJmuiden,Iserlohn,Bochum,1] + v[IJmuiden,Neuss,Bochum,1] + v[IJmuiden,Schwerte,Bochum,1] + v[IJmuiden,IJmuiden,Bochum,1])*3*(0*v[IJmuiden,Bochum,Bochum,1] + 55*v[IJmuiden,Boenen,Bochum,1] + 21*v[IJmuiden,Dortmund,Bochum,1] + 19*v[IJmuiden,Gelsenkirchen,Bochum,1] + 41*v[IJmuiden,Hagen,Bochum,1] + 51*v[IJmuiden,Iserlohn,Bochum,1] + 56*v[IJmuiden,Neuss,Bochum,1] + 39*v[IJmuiden,Schwerte,Bochum,1] + 250*v[IJmuiden,IJmuiden,Bochum,1] + 250*v[IJmuiden,Bochum,IJmuiden,1] + 282*v[IJmuiden,Boenen,IJmuiden,1] + 266*v[IJmuiden,Dortmund,IJmuiden,1] + 234*v[IJmuiden,Gelsenkirchen,IJmuiden,1] + 289*v[IJmuiden,Hagen,IJmuiden,1] + 299*v[IJmuiden,Iserlohn,IJmuiden,1] + 259*v[IJmuiden,Neuss,IJmuiden,1] + 279*v[IJmuiden,Schwerte,IJmuiden,1]) + (v[IJmuiden,Bochum,Bochum,2] + v[IJmuiden,Boenen,Bochum,2] + v[IJmuiden,D

In [11]:
# Print all variables that are not null (non-zero) in the model

# Iterate over binary variables (e.g., v, z) and print if they are 1
print("Binary Variables (Not null):")
for f in model3.F:
    for i in model3.A_f[f]:
        for j in model3.A_f[f]:
            for t in model3.T:
                if pe.value(model3.v[(f, i, j, t)]) > 0.5:  # If v is 1
                    print(f"v[{f}, {i}, {j}, {t}] = {pe.value(model3.v[(f, i, j, t)])}")

    # For z variables
    for a in model3.A:
        for t in model3.T:
            if pe.value(model3.z[f, a, t]) > 0.5:  # If z is 1
                print(f"z[{f}, {a}, {t}] = {pe.value(model3.z[f, a, t])}")

# Iterate over continuous variables (e.g., q, t) and print if they are non-zero
print("\nContinuous Variables (Not null):")
for f in model3.F:
    for a in model3.A:
        for t in model3.T:
            for r in model3.R:
                if pe.value(model3.q[r, f, a, t]) > 1e-6:  # If q is not zero
                    print(f"q[{r}, {f}, {a}, {t}] = {pe.value(model3.q[r, f, a, t])}")

            # For t variable (shipments)
            if pe.value(model3.t[f, a, t]) > 1e-6:  # If t is non-zero
                print(f"t[{f}, {a}, {t}] = {pe.value(model3.t[f, a, t])}")

# You can also do this for other variables like i, u, etc.


Binary Variables (Not null):
v[IJmuiden, Neuss, IJmuiden, 1] = 1.0
v[IJmuiden, IJmuiden, Neuss, 1] = 1.0
z[IJmuiden, Neuss, 1] = 1.0
v[Segal, Boenen, Segal, 1] = 1.0
v[Segal, Hagen, Boenen, 1] = 1.0
v[Segal, Segal, Hagen, 1] = 1.0
z[Segal, Boenen, 1] = 1.0
z[Segal, Hagen, 1] = 1.0
v[South Wales, Bochum, Hagen, 3] = 1.0
v[South Wales, Bochum, South Wales, 1] = 1.0
v[South Wales, Boenen, South Wales, 2] = 1.0
v[South Wales, Dortmund, Bochum, 1] = 1.0
v[South Wales, Gelsenkirchen, Iserlohn, 1] = 1.0
v[South Wales, Hagen, Boenen, 2] = 1.0
v[South Wales, Hagen, South Wales, 3] = 1.0
v[South Wales, Iserlohn, Hagen, 2] = 1.0
v[South Wales, Iserlohn, Schwerte, 1] = 1.0
v[South Wales, Neuss, Bochum, 3] = 1.0
v[South Wales, Schwerte, Dortmund, 1] = 1.0
v[South Wales, South Wales, Gelsenkirchen, 1] = 1.0
v[South Wales, South Wales, Iserlohn, 2] = 1.0
v[South Wales, South Wales, Neuss, 3] = 1.0
z[South Wales, Bochum, 1] = 1.0
z[South Wales, Bochum, 3] = 1.0
z[South Wales, Boenen, 2] = 1.0
z[South 